In [0]:
import numpy as np
import pandas as pd
import cv2
import os
from google.colab import drive
#_____________________________________________________________________________________________________
drive.mount('/content/drive')

path        = "/content/drive/My Drive/ATI/"
img_Prepro  = "/content/drive/My Drive/ATI/Obj_dete_Prepro/"

train_csv = path + "train.csv"
test_csv  = path + "test.csv"
csv_path  = [train_csv , test_csv]

HEIGHT = 49
WIDTH = 37
#_____________________________________________________________________________________________________
def image_preprocessing(image_path,y1,x1,y2,x2):
    image = cv2.imread(images_path)
    h, w, c = image.shape
    # find scale
    S_h = HEIGHT/h
    S_w = WIDTH/w
    image = image.astype(np.float32, copy=True)
    image -= np.array([[[102.9801, 115.9465, 122.7717]]])
    
    mask = np.zeros(image.shape, dtype=np.uint8)
    blurred_img = cv2.GaussianBlur(image, (21, 21), 0)
    mask = cv2.rectangle(mask, (x1, y1), (x2, y2), (255,255,255), -1) 
    result = np.where(mask==np.array([255, 255, 255]), image, blurred_img)
    result = cv2.resize(result,(WIDTH,HEIGHT))

    return result,S_h,S_w  
#_____________________________________________________________________________________________________
#(y1, x1 , y2, x2)
def spatial_vector(y1,x1,y2,x2, S_h, S_w):    
    y = ((y1 + y2) / 2.0) * S_w
    x = ((x1 + x2) / 2.0) * S_h
    h = (y2 - y1) * S_w
    w = (x2 - x1) * S_h

    spatial_vector = [x, y , w , h , w* h , w/37 , h/49,      
                         np.log(x), np.log(y), np.log(w), 
                         np.log(h),np.log(w/37) , np.log(h/49) , np.log(w/h)]
        
    return spatial_vector

#_____________________________________________________________________________________________________

for r in range(2):
    csv_path_use  = csv_path[r]
    df_use        = pd.read_csv(csv_path_use)
    spatial_vec = []
    print("csv_path_use :", csv_path_use)
    
    temp = []
    for i in range(df_use.shape[0]) :
        print("[{0}/{1}] ".format(i ,df_use.shape[0] ))
        images_path   = df_use["image_path"][i]
        y1 = df_use['y1'][i]
        x1 = df_use['x1'][i]
        y2 = df_use['y2'][i]
        x2 = df_use['x2'][i]
        
        image, S_h, S_w = image_preprocessing(images_path, y1,x1,y2,x2)
        #image = cv2.imwrite(images_path , image)
        
        temp["spatial_vec"] = spatial_vector(y1,x1,y2,x2, S_h, S_w)
        spatial_vec.append(temp)
    
    df_spatial_vec = pd.DataFrame(spatial_vec)
    train_csv.join(df_spatial_vec)